## Calculating Transfer Function between two TE accelerometers with a labJack T7-Pro
This is an attempt to calculate the transfer function from the stored\
accelerations of two accelerometers.  It estimates the TF as the ratio of the\
cross spectral densities H(k) = Pyy(k) / Pxy(k), as in this link:\
https://www.mathworks.com/help/signal/ref/tfestimate.html#bufqg8e

Craig Lage - Sep 27, 2021


In [ ]:
import sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft, fftfreq, ifft, irfft
from scipy.interpolate import splprep, splrep, splev
from scipy.signal import TransferFunction as tf
from scipy.signal import dlsim, csd, welch
from scipy.optimize import minimize

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
# Read in the pickled accelerometer data
scanRate = 1000
readTime = 10.0
df = pd.read_pickle("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6.pkl")

In [ ]:
# Write to a text file
filename = "/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6.txt"
file = open(filename, "w")
file.write("Time(s) \t  Input (g) \t Output(g)\n")
t = 0.0
deltaT = .001
for i in range(len(df.index)):
    file.write(f"{t:.3f} \t \t {df['a1z'][df.index[i]]:.6f} \t {df['a2z'][df.index[i]]:.6f}\n ")
    t += deltaT
file.close()

In [ ]:
# Check to make sure it looks the same.
data = np.loadtxt(filename, skiprows=1)
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)

plt.subplot(1,2,1)
plt.plot(data[450:750,0], data[450:750,1])
plt.subplot(1,2,2)
plt.plot(data[450:750,0], data[450:750,2])


In [ ]:
# Plot the data at different time magnifications.
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)

for n, scanIndex in enumerate([[0, int(scanRate*readTime-1)], [450, 750], [500,600]]):

    sub_df = df[df.index[scanIndex[0]] : df.index[scanIndex[1]]]
    
    plt.subplot(2,3,n+1)
    ax1 = sub_df['a1z'].plot(label="a1z", color='red')
    ax1.set_title("Axis 1", fontsize=16)
    ax1.set_ylabel("Acceleration(g)")
    ax1.legend(loc='center left')

    plt.subplot(2,3,n+4)
    ax2 = sub_df['a2z'].plot(label="a2z", color='red')
    ax2.set_title("Axis 2", fontsize=16)
    ax2.set_ylabel("Acceleration(g)")
    ax2.legend(loc='center left')
    
    if n == 0: # save this data for the FFT
        a1z = np.array(sub_df['a1z'].tolist())
        a2z = np.array(sub_df['a2z'].tolist())

#plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6_14Sep21.pdf")